This is a user app that leverages Tetration Alfred in order to annotate discovered endpoints with ACI details

### Configuration

In [40]:
data_tap = 'datatap_name_here'
# Uncomment and specify a topic for TetrationOS < 2.2.1.31
# topic = 'kafka_topic_here'
ip_subnet_prefix = 'x.y_here'

## -----------------------------------------------

In [41]:
# Import required libraries
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import pandas as pd
import json

In [42]:
# Fetch IPs from inventory and dump into a Pandas dataset. Use ip_address LIKE to limit scope to ACI endpoints 
inventory_data = sc._jvm.com.tetration.apps.IO.read(sqlContext._ssql_ctx, "/tetration/inventory/", "PARQUET", "LASTHOUR")
inventory_data.registerTempTable("inventory1h")
query = "select ip_address from inventory1h where ip_address LIKE " + "'%{}%'".format(ip_subnet_prefix)
inventory_1h_ips = sqlContext.sql(query)
inventory1hds = inventory_1h_ips.toPandas()

In [43]:
# Create the list of endpoints
endpoint_list = []
for endpoint in inventory1hds.ip_address.unique():
    endpoint_list.append(endpoint)

In [44]:
# Build question JSON
d = {'query': 'get_endpoint_detail',
     'payload' : endpoint_list}
json_question = json.dumps(d)

In [45]:
# Send a Kafka message with the question
# Use the commented command when using Tetration OS < 2.2.1.31
# sc._jvm.com.tetration.apps.DataTaps.sendMessage(json_question, data_tap, topic)
sc._jvm.com.tetration.apps.DataTaps.sendMessage(json_question, data_tap)

1